# RNA-seq calling and QC

This document shows the use of various modules to prepare reference data, perform RNA-seq calling, expression level quantification and quality control. In particular,

1. [`reference_data.ipynb`](../data_preprocessing/reference_data.html)
2. [`RNA_calling.ipynb`](calling/RNA_calling.html)
3. [`readCount_QC.ipynb`](QC/readCount_QC.html)

A minimal working example is available on [Google Drive](https://drive.google.com/drive/u/0/folders/11kQv7PXozsKkgeqADH-28bC_kZ-w_oHo).

This is a master control notebook for the molecular_phenotype_calling section of our pipeline. When following example commands was excuted, it will checked if the reference data was presented and generate the reference data if not. After that, molecular phenotype calling will be done based on the recipe that were fed into it. For each rows in the recipe, It will output a bed.gz file that are ready for downstream analysis.

By default, the reference data is assumed to be stored in the reference_data folder under the cwd. However, user can specify whether the 

- Input:

    A recipe file, with columns: `"Theme", "Sample_fastq_list", "Data_dir"`

- Output: 
    
    A bed.gz file for each row in the Input
  
- parameters: 
    
    - flag `--is-stranded` or `--no-is-stranded` : whether the rna seq data intended to be called is stranded or not. 
    - flag `--pair-ended` or `--no-pair-ended` : whether the rna seq data intended to be called is paired-ended or not.
    - `--exe_dir` : diretory to the workflow notebook
    - `--sample-participant-lookup` : a sample participant lookup table with the first column being sample ID and second column being the cooresponding sample ids in the genotype file

In [ ]:
[global]
## The aforementioned input recipe
parameter: recipe = path
## Overall wd, the file structure of analysis is wd/[steps]/[sub_dir for each steps]
parameter:  cwd = path(".")
## dir to store the reference data
parameter: refd = f'{cwd}/reference_data/'

## Diretory to the excutable
parameter: exe_dir = path("~/GIT/xqtl-pipeline/")
parameter: container = '/mnt/mfs/statgen/containers/twas_latest.sif'
parameter: container_base_bioinfo = '/mnt/mfs/statgen/containers/bioinfo.sif'
parameter: container_apex = '/mnt/mfs/statgen/containers/apex.sif'
parameter: container_PEER = '/mnt/mfs/statgen/containers/PEER.sif'
parameter: container_TensorQTL = '/mnt/mfs/statgen/containers//TensorQTL.sif'
parameter: container_rnaquant = '/mnt/mfs/statgen/containers/rna_quantification.sif'
parameter: container_flashpca = '/mnt/mfs/statgen/containers/flashpcaR.sif'
parameter: sample_participant_lookup = path
parameter: phenotype_id_type = "gene_name" 
parameter: yml = path("csg.yml") 
import pandas as pd
input_inv = pd.read_csv(recipe, sep = "\t").to_dict("records")
import os

## Download, Process, and index reference data (first time use)

### Download reference and annotation data (first time use)

In [ ]:
[ref_data_1,download_reference_data]
output: f'{refd:a}/GRCh38_full_analysis_set_plus_decoy_hla.fa',
        f"{refd:a}/Homo_sapiens.GRCh38.103.chr.gtf",
        f"{refd:a}/ERCC92.gtf", f"{refd:a}/ERCC92.fa"
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
        sos run $[exe_dir]/pipeline/reference_data.ipynb download_hg_reference --cwd $[_output[0]:d]
        sos run $[exe_dir]/pipeline/reference_data.ipynb download_gene_annotation --cwd $[_output[0]:d]
        sos run $[exe_dir]/pipeline/reference_data.ipynb download_ercc_reference --cwd $[_output[0]:d]

### Reformat reference and annotation data (first time use)

In [ ]:
[ref_data_2]
output:  f'{_input[0]:n}.noALT_noHLA_noDecoy_ERCC.fasta',f'{_input[0]:bn}.noALT_noHLA_noDecoy_ERCC.dict'
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
        sos run $[exe_dir]/reference_data.ipynb hg_reference \
            --cwd $[_output[0]:d] \
            --ercc-reference $[_input[3]] \
            --hg-reference $[_input[1]] \
            --container $[container_rnaquant]

In [ ]:
**Notice: you need to know if your RNA-seq data is stranded or unstranded. If it is stranded, you should add `--is-stranded` to the command below.**

In [ ]:
[gene_annotation]
parameter: is_stranded = bool
input: output_from(ref_data_1),output_from(ref_data_2)[0]
output: f'{_input[1]:n}.reformatted.ERCC.gtf', f'{_input[1]:n}.reformatted.{".collapse_only" if is_stranded else ""}.gene.ERCC.gtf'
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    sos run $[exe_dir]/reference_data.ipynb gene_annotation \
            --cwd $[_output[0]:d] \
            --ercc-gtf $[_input[2]] \
            --hg-gtf $[_input[1]] \
            --hg-reference $[_input[4]] \
            --container $[container_rnaquant] \
            $[--is-stranded if is_stranded else --no-is-stranded]

### Generation of Index (First time only)

In [ ]:
[indexing]
input: output_from(gene_annotation)[1],output_from(ref_data_2)[0]
output: f"{refd:a}/RSEM_Index/rsem_reference.n2g.idx.fa", 
        f"{refd:a}/RSEM_Index/rsem_reference.grp", 
        f"{refd:a}/RSEM_Index/rsem_reference.idx.fa",
        f"{refd:a}/RSEM_Index/rsem_reference.ti", 
        f"{refd:a}/RSEM_Index/rsem_reference.chrlist",
        f"{refd:a}/RSEM_Index/rsem_reference.seq", 
        f"{refd:a}/RSEM_Index/rsem_reference.transcripts.fa",
        f"{refd:a}/STAR_Index/chrName.txt",
        f"{refd:a}/STAR_Index/Log.out", 
        f"{refd:a}/STAR_Index/transcriptInfo.tab",
        f"{refd:a}/STAR_Index/exonGeTrInfo.tab", 
        f"{refd:a}/STAR_Index/SAindex",
        f"{refd:a}/STAR_Index/SA",
        f"{refd:a}/STAR_Index/genomeParameters.txt", 
        f"{refd:a}/STAR_Index/chrStart.txt",
        f"{refd:a}/STAR_Index/sjdbList.out.tab", 
        f"{refd:a}/STAR_Index/exonInfo.tab",
        f"{refd:a}/STAR_Index/sjdbList.fromGTF.out.tab", 
        f"{refd:a}/STAR_Index/chrLength.txt",
        f"{refd:a}/STAR_Index/sjdbInfo.txt", 
        f"{refd:a}/STAR_Index/Genome",
        f"{refd:a}/STAR_Index/chrNameLength.txt", 
        f"{refd:a}/STAR_Index/geneInfo.tab"
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
sos run $[exe_dir]/reference_data.ipynb STAR_index RSEM_index \
    --cwd $[refd:a] \
    --hg-reference $[_input[1]] \
    --hg-gtf $[_input[0]] \
    --container $[container_rnaquant] \
    --mem 40G

## Perform data quality summary via `fastqc`

**You need to figure out from fastqc results what adapter reference sequence to use for `--fasta_with_adapters_etc`. See section Step 1 below for details**. We chose `--fasta_with_adapters_etc TruSeq3-PE.fa` for our toy example.

* `fasta_with_adapters_etc`: **filename** for the adapter reference file. According to `Trimmomatic` documention,

> As a rule of thumb newer libraries will use `TruSeq3`, but this really depends on your service provider. If you use FASTQC, the "Overrepresented Sequences" report can help indicate which adapter file is best suited for your data. "Illumina Single End" or "Illumina Paired End" sequences indicate single-end or paired-end `TruSeq2` libraries, and the appropriate adapter files are `TruSeq2-SE.fa` and `TruSeq2-PE.fa` respectively. "TruSeq Universal Adapter" or "TruSeq Adapter, Index …" indicates `TruSeq-3` libraries, and the appropriate adapter files are `TruSeq3-SE.fa` or `TruSeq3-PE.fa`, for single-end and paired-end data respectively. Adapter sequences for `TruSeq2` multiplexed libraries, indicated by "Illumina Multiplexing 
…", and the various RNA library preparations are not currently included.

We have `fastqc` workflow defined as followed. Users should decide what fasta adapter reference to use based on `fastqc` results (or their own knowledge).
It is assumed that all studies that were ran in one run are using the same adapter file




In [ ]:
[fastqc]
input: for_each = "input_inv"
output: f'{cwd:a}/output/rnaseq/fastqc/'
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    sos run $[exe_dir]/RNA_calling.ipynb fastqc \
        --cwd $[_output[0]:d] \
        --samples $[_input_inv['Sample_fastq_list']] \
        --data-dir $[_input_inv['Data_dir']]   \
        --container $[container_rnaquant]

## Multi-sample RNA-seq Calling, QC, and normalizations (Automatically for every studies using the same set of ref/index/gtf)

### Call gene-level RNA expression via `rnaseqc`

In [ ]:
[rnaseqc]
parameter:fasta_with_adapters_etc = "TruSeq3-PE.fa"
input:  output_from("gene_annotation")[-1],output_from("indexing")[-1],for_each = "input_inv"
output: 
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
sos run RNA_calling.ipynb rnaseqc_call \
    --cwd $[_output[0]:d]/ \
    --samples  $[_input_inv['Sample_fastq_list']] \
    --data-dir $[_input_inv['Data_dir']] \
    --fasta_with_adapters_etc $[fasta_with_adapters_etc]  \
    --STAR-index $[_input[1]:d] \
    --gtf $[_input[0]] \
    --container $[container_rnaquant] \
    --mem 40G

### Call transcript level RNA expression via `RSEM`

In [ ]:
[RSEM]
parameter:fasta_with_adapters_etc = "TruSeq3-PE.fa"
input:  output_from("indexing")[0],output_from("indexing")[-1],for_each = "input_inv"
output: 
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
sos run RNA_calling.ipynb rnaseqc_call \
    --cwd $[_output[0]:d]/ \
    --samples  $[_input_inv['Sample_fastq_list']] \
    --data-dir $[_input_inv['Data_dir']] \
    --fasta_with_adapters_etc $[fasta_with_adapters_etc]  \
    --STAR-index $[_input[1]:d] \
    --RSEM-index $[_input[0]:d] \
    --container $[container_rnaquant] \
    --mem 40G

## Multi-sample RNA-seq QC (For every set of samples)

We need to use a different MWE data-set that contains multiple samples -- here is the [Google Drive link](https://drive.google.com/drive/u/0/folders/1Rv2bWHBbX_tastTh49ToYVDMV6rFP5Wk).

In [ ]:
[QC]
input: output_from(""),output_from("QC")
output: 
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'

    sos run bulk_expression_QC.ipynb qc \
        --cwd output \
        --tpm-gct data/mwe.TPM.gct \
        --counts-gct data/mwe.Counts.gct \
        --container container/rna_quantification.sif

## Multi-sample read count normalization (For every set of samples)

In [ ]:
[normalization]
parameter: sample-participant-lookup = path
input: output_from("QC"),output_from("gene_annotation")
output: 
bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
sos run bulk_expression_normalization.ipynb normalize \
    --cwd $[_output:d] \
    --tpm-gct $[_input[0]] \
    --counts-gct $[_input[1]] \
    --annotation-gtf  $[_input[-1]]  \
    --sample-participant-lookup $[sample-participant-lookup] \
    --container $[container_rnaquant]  \
    --count-threshold 1